In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from textattack.augmentation import EasyDataAugmenter
warnings.filterwarnings('ignore')
df = pd.read_csv('sahte_is.csv')
df.head()

,job_title,company_name,company_desc,job_desc,job_requirement,salary,location,employment_type,department,label
0,Accountant,Equity Model Limited,"Accounting, Auditing & Finance","Compiling, analyzing, and reporting financial ...",This position is open preferably to a male can...,"75,000 - 150,000",Abuja,Full Time,Law & Compliance,0
1,Content Writer,CLINTON FUND (CF),Management & Business Development,"Creating, improving and maintaining content to...","Bachelor's degree in Journalism, English, Com...","60,000 - 100,000",Lagos,Full Time,Content Writing,1
2,Accountant,Schleez Nigeria Limited,"Accounting, Auditing & Finance","Managing financial transactions, preparing fin...",Minimum of Bachelor's degree in Accounting or ...,Negotiable,"First Floor, Left Wing, No. 49, Olowu Street, ...",Full-time,Accounting,1
3,Sales Executive,Bons Industries Limited,Marketing & Communications,"Understanding of the sales process and dynamics.""",Minimum academic qualification of BSC/HND Degr...,"75,000 - 150,000",Enugu,Full Time,Manufacturing & Warehousing,0
4,Bulk/Partnership Marketing Officer,TAMAK LOGISTICS,Marketing & Communications,Establish relationships with major businesses ...,"Be smart & resourceful.,Great knowledge of how...","Less than 75,000",Lagos,Full Time,Shipping & Logistics,0


In [3]:


df.isnull().sum()# boş değer var mı yok mu ona baktım hepsi dolu gözüküyor.

,0
job_title,0
company_name,0
company_desc,0
job_desc,0
job_requirement,0
salary,0
location,0
employment_type,0
department,0
label,0


In [4]:
df.info()#veri içeriğine baktım.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        202 non-null    object
 1   company_name     202 non-null    object
 2   company_desc     202 non-null    object
 3   job_desc         202 non-null    object
 4   job_requirement  202 non-null    object
 5   salary           202 non-null    object
 6   location         202 non-null    object
 7   employment_type  202 non-null    object
 8   department       202 non-null    object
 9   label            202 non-null    int64 
dtypes: int64(1), object(9)
memory usage: 15.9+ KB


In [5]:
df.shape

(202, 10)

In [6]:
df.columns

Index(['job_title', 'company_name', 'company_desc', 'job_desc',
       'job_requirement', 'salary', 'location', 'employment_type',
       'department', 'label'],
      dtype='object')

In [7]:
pd.set_option('display.max_columns', None)  # Tüm sütunları göster
pd.set_option('display.max_rows', None)     # Tüm satırları göster
pd.set_option('display.max_colwidth', None)

In [8]:

df["company_name"]=df["company_name"].str.upper()

# company_name verilerinde büyük/küçük harf uyumu konusunda sorunlar vardı burada o sorunu çözdük

In [9]:
from transformers import pipeline

summarizer = pipeline("summarization")

def transformers_summarize(text):
    summary = summarizer(text, max_length=30, min_length=10, do_sample=False)
    return summary[0]['summary_text']

df['augmented_job_desc'] = df['job_desc'].apply(transformers_summarize)

# bu kısımla beraber job_desc verimizin açıklamalarını ekledik

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your max_length is set to 30, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
Your max_length is set to 30, but your input_length is only 17. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 30, but your input_length is only 10. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)
Your max_length is s

In [20]:
import pandas as pd
from textattack.augmentation import EasyDataAugmenter
from deep_translator import GoogleTranslator

# CSV dosyasını okuma
df = pd.read_csv('sahte_is.csv')

# TextAttack veri artırıcıyı oluşturma
augmenter = EasyDataAugmenter()

# Çevirici nesnesi oluşturma
translator = GoogleTranslator()

# Veri artırımı fonksiyonu
def augment_text(text):
    if isinstance(text, str):
        return augmenter.augment(text)
    return text

# İstenmeyen verileri filtreleme fonksiyonu
def filter_augmented_text(text):
    if 'lxxv' in text.lower():
        return text.replace('lxxv', '75')
    return text

# Çeviri yapacak fonksiyon
def translate_text(text, dest='tr'):
    if not isinstance(text, str) or not text.strip():
        return ""
    try:
        translation = translator.translate(text, target=dest)
        return translation
    except Exception as e:
        print(f"Error during translation: {e}")
        return ""

# Augment yapmak istediğiniz sütun adlarını liste halinde oluşturma
columns_to_augment = ['job_title', 'salary', 'job_desc']

# Veri artırımı ve filtreleme işlemini yapma
for column in columns_to_augment:
    # Metinlerin artırılması
    df[f'{column}_augmented'] = df[column].apply(lambda x: augment_text(x))

    # Filtreleme işlemi
    df[f'{column}_augmented'] = df[f'{column}_augmented'].apply(lambda x: [filter_augmented_text(y) for y in x] if isinstance(x, list) else x)

    # Augmented metinleri Türkçeye çevirme
    # Dikkat: Eğer x liste değilse, çeviri işlemi doğrudan uygulanır
    df[f'{column}_augmented_translated'] = df[f'{column}_augmented'].apply(
        lambda x: [translate_text(y) for y in x] if isinstance(x, list) else translate_text(x)
    )

# Artırılmış ve çevrilmiş verileri yeni bir CSV dosyasına kaydetme
df.to_csv('metin_verisi_artirilmis_ve_cevrilmis.csv', index=False)

df.head(20)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,job_title,company_name,company_desc,job_desc,job_requirement,salary,location,employment_type,department,label,job_title_augmented,job_title_augmented_translated,salary_augmented,salary_augmented_translated,job_desc_augmented,job_desc_augmented_translated
0,Accountant,Equity Model Limited,"Accounting, Auditing & Finance","Compiling, analyzing, and reporting financial data.","This position is open preferably to a male candidate,Minimum academic qualification preferably in accounting,Candidates aged between 22 - 30 years,Ability to write clearly and help with word processing when necessary.,A warm personality and strong communication skills.,Ability to work well under limited supervision.,Great communication and writing skills.,Hands-on experience in managing time and personnel.","75,000 - 150,000",Abuja,Full Time,Law & Compliance,0,"[Accountant, comptroller Accountant, controller]","[Accountant, comptroller Accountant, controller]","[75,000 -,000, 75,000 - 000,150, seventy-five,000 - 150,000, 75,000 - 150,cl 000]","[75,000 -,000, 75,000 - 000,150, seventy-five,000 - 150,000, 75,000 - 150,cl 000]","[compile, analyzing, and reporting financial data., Compiling, analyzing, and reporting financial datum data., Compiling, data, and reporting financial analyzing., , analyzing, and reporting financial data.]","[compile, analyzing, and reporting financial data., Compiling, analyzing, and reporting financial datum data., Compiling, data, and reporting financial analyzing., , analyzing, and reporting financial data.]"
1,Content Writer,CLINTON FUND (CF),Management & Business Development,"Creating, improving and maintaining content to achieve our business goals. Your duties will also include sharing content to raise brand awareness and monitoring web traffic and metrics to identify best practices.","Bachelor's degree in Journalism, English, Communications or related field.","60,000 - 100,000",Lagos,Full Time,Content Writing,1,"[Content message Writer, Writer, Writer Content, content Writer]","[Content message Writer, Writer, Writer Content, content Writer]","[60,000 -,000, 100,000 - 60,000, LX,000 - 100,000, 60,sixty 000 - 100,000]","[60,000 -,000, 100,000 - 60,000, LX,000 - 100,000, 60,sixty 000 - 100,000]","[Creating, improving and maintaining content to achieve our business goals. Your duties will also include sharing content to nurture brand cognisance and monitoring web traffic and prosody to identify best practices. , Creating, improving maintaining content to achieve our business goals. Your duties will also sharing content raise brand awareness and monitoring web traffic and metrics to identify best practices. , Creating, metrics and maintaining content to achieve our business goals. Your duties will also include sharing content monitoring raise brand awareness and to web traffic and improving to identify best practices. , Creating, improving and good maintaining content to achieve our business goals. Your duties will also include sharing content besides to raise brand awareness and monitoring web traffic and metrics to identify best practices. ]","[Creating, improving and maintaining content to achieve our business goals. Your duties will also include sharing content to nurture brand cognisance and monitoring web traffic and prosody to identify best practices., Creating, improving maintaining content to achieve our business goals. Your duties will also sharing content raise brand awareness and monitoring web traffic and metrics to identify best practices., Creating, metrics and maintaining content to achieve our business goals. Your duties will also include sharing content monitoring raise brand awareness and to web traffic and improving to identify best practices., Creating, improving and good maintaining content to achieve our business goals. Your duties will also include sharing content besides to raise brand awareness and monitoring web traffic and metrics to identify best practices.]"
2,Accountant,Schleez Nigeria Limited,"Acc

In [21]:
from deep_translator import GoogleTranslator

# Çeviri yapacak fonksiyon
def translate_text(text, dest='tr'):
    if not text or not isinstance(text, str):
        return ""
    try:
        translator = GoogleTranslator(target=dest)
        translation = translator.translate(text)
        return translation
    except Exception as e:
        print(f"Error during translation: {e}")
        return ""

# Test çevirisi
print(translate_text("Hello, world!"))

Selam Dünya!


In [25]:
import pandas as pd
from textattack.augmentation import EasyDataAugmenter
from deep_translator import GoogleTranslator

# CSV dosyasını okuma
df = pd.read_csv('sahte_is.csv')

# TextAttack veri artırıcıyı oluşturma
augmenter = EasyDataAugmenter()

# Çevirici nesnesi oluşturma
translator = GoogleTranslator(target='tr')

# Veri artırımı fonksiyonu
def augment_text(text):
    return augmenter.augment(text)

# İstenmeyen verileri filtreleme fonksiyonu
def filter_augmented_text(text):
    if 'lxxv' in text.lower():
        return text.replace('lxxv', '75')
    return text

# Çeviri yapacak fonksiyon
def translate_text(text):
    if not text or not isinstance(text, str):
        return ""
    try:
        translation = translator.translate(text)
        return translation
    except Exception as e:
        print(f"Error during translation for text '{text}': {e}")
        return ""

# Augment ve çeviri yapmak istediğiniz sütun adlarını liste halinde oluşturma
columns_to_augment_and_translate = ['job_title', 'salary', 'job_desc']

# Veri artırımı, filtreleme ve çeviri işlemini yapma
for column in columns_to_augment_and_translate:
    # Augment işlemi
    df[f'{column}_augmented'] = df[column].apply(lambda x: augment_text(x))
    df[f'{column}_augmented'] = df[f'{column}_augmented'].apply(lambda x: [filter_augmented_text(y) for y in x])

    # Çeviri işlemi
    df[f'{column}_augmented_translated'] = df[f'{column}_augmented'].apply(
        lambda x: [translate_text(y) for y in x]
    )

# Artırılmış ve çevrilmiş verileri yeni bir CSV dosyasına kaydetme
df.to_csv('metin_verisi_artirilmis_ve_cevrilmis.csv', index=False)

print(df.head(20))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                             job_title                         company_name  \
0                           Accountant                 Equity Model Limited   
1                       Content Writer                    CLINTON FUND (CF)   
2                           Accountant              Schleez Nigeria Limited   
3                      Sales Executive              Bons Industries Limited   
4   Bulk/Partnership Marketing Officer                      TAMAK LOGISTICS   
5             Marketer/Sales Executive                  Color Sense Limited   
6         Medical Sales Representative                           Sylken Ltd   
7                     Group Sales Lead                    Leadway Assurance   
8                    Real Estate Agent                  HOME PLACES CONSULT   
9         Medical Sales Representative                           Sylken Ltd   
10                          Accountant                        MPG RESOURCES   
11        Medical Sales Representative              

In [27]:
df = pd.read_csv('metin_verisi_artirilmis_ve_cevrilmis.csv')
df.head()
df.drop(columns=['job_title_augmented', 'salary_augmented', 'job_desc_augmented'], inplace=True)
df.head()

,job_title,company_name,company_desc,job_desc,job_requirement,salary,location,employment_type,department,label,job_title_augmented_translated,salary_augmented_translated,job_desc_augmented_translated
0,Accountant,Equity Model Limited,"Accounting, Auditing & Finance","Compiling, analyzing, and reporting financial data.","This position is open preferably to a male candidate,Minimum academic qualification preferably in accounting,Candidates aged between 22 - 30 years,Ability to write clearly and help with word processing when necessary.,A warm personality and strong communication skills.,Ability to work well under limited supervision.,Great communication and writing skills.,Hands-on experience in managing time and personnel.","75,000 - 150,000",Abuja,Full Time,Law & Compliance,0,"['Muhasebeci', 'kontrolör Muhasebeci', 'denetçi']","[',000 - 150.000', '75.000 - 150.000 TL', '75.000 - 000.150', '75.000 - 150.000']","[', finansal verileri analiz etmek ve raporlamak.', 'Finansal verilerin derlenmesi, analiz edilmesi ve bilgi raporlanması.', 'Finansal verilerin derlenmesi, analiz edilmesi ve muhasebeleştirilmesi.', 've, analiz etmek, raporlamak, finansal verileri derlemek.']"
1,Content Writer,CLINTON FUND (CF),Management & Business Development,"Creating, improving and maintaining content to achieve our business goals. Your duties will also include sharing content to raise brand awareness and monitoring web traffic and metrics to identify best practices.","Bachelor's degree in Journalism, English, Communications or related field.","60,000 - 100,000",Lagos,Full Time,Content Writing,1,"['yazar İçerik Yazarı', 'Yazar', 'Yazar İçeriği', 'İçerik yazarı']","['000,60 - 100.000', '60.000 -,000', 'altmış.000 - 100.000', '60.000 - 100.000']","['İş hedeflerimize ulaşmak için trafiği oluşturmak, iyileştirmek ve sürdürmek. Görev farkındalığınız ayrıca marka isteğini yükseltmek için içerik paylaşmayı ve en iyi uygulamaları belirlemek için web içeriğini ve ölçümlerini izlemeyi de içerir.', 'İşimizi başarmak için içerik oluşturma, iyileştirme ve sürdürme. Görevleriniz arasında marka bilinirliğini artırmak için paylaşım yapmak ve en iyi uygulamaları belirlemek için web trafiğini ve ölçümlerini izlemek yer alacaktır.', 'İş hedeflerimize ulaşmak için içerik oluşturma, iyileştirme ve sürdürme. Görevleriniz arasında marka bilinirliğini artırmak için içerik paylaşma ve en iyi uygulamaları belirlemek için web trafiğini ve ölçümlerini izleme de yer alacaktır.', 'İş hedeflerimize ulaşmak için içerik oluşturma, geliştirme ve kullanma. Görevleriniz arasında marka bilinirliğini artırmak için içerik paylaşma ve doğru uygulamaları belirlemek için web trafiğini ve ölçümlerini izleme yer alacaktır.']"
2,Accountant,Schleez Nigeria Limited,"Accounting, Auditing & Finance","Managing financial transactions, preparing financial reports, and providing financial advice to clients.","Minimum of Bachelor's degree in Accounting or related field, Professional certification such as ACCA or ICAN, Proven experience as an Accountant, Strong computer skills, including proficiency in Excel, Excellent communication and interpersonal skills",Negotiable,"First Floor, Left Wing, No. 49, Olowu Street, Off Moboloji Bank, Anthony Way, Ikeja, Lagos State",Full-time,Accounting,1,"['Muhasebeci', 'muhasebeci', 'denetçi Muhasebeci']","['taşınabilir', 'Pazarlık edilebilir', 'taşınabilir, pazarlığa tabi']","['Müşterilere finansal işlemleri yönetmek, finansal raporlar hazırlamak ve finansal danışmanlık sağlamak.', 'Finansal işlemleri yönetmek, finansal raporlar hazırlamak ve müşterilere finansal destek sağlamak.', 'Müşterilere finansal işlemleri yönetmek, finansal raporlar hazırlamak, mali konularda danışmanlık yapmak.', 'Finansal işlemleri yönetmek, finansal raporlar hazırlamak ve müşterilere finansal danışmanlık sağlamak.']"
3,Sales Executive,Bons Industries Limited,Marketing & Communications,"Understanding of the sales process and dynamics.""","Minimum academic qualification of BSC/HND Degree in 